# ProgPy Tutorial
_2024 PHM Society Conference_
_November, 2024_

Please put questions in the Whova App or raise your hand.

# Pre-Work
_We recommend installing ProgPy prior to the tutorial_

The latest stable release of ProgPy is hosted on PyPi. To install via the command line, use the following command: 

`$ pip install progpy`

The documentation for ProgPy can be found [here](https://nasa.github.io/progpy/index.html). We will reference this material throughout the tutorial. ProgPy can be found on GitHub at [this link](https://github.com/nasa/progpy).

Please download the Whova App (<span style="color:red"> link? </span>) for live Q&A during the session.

Next, lets download the data we will be using for this tutorial. To do this we will use the datasets subpackage in progpy.

In [ ]:
from progpy.datasets import nasa_battery
(desc, data) = nasa_battery.load_data(1)

## Introduction to ProgPy

<span style="color:red">
•	Put questions in the whova app
•	What it is
•	Where to find it
•	Installing it – Pre-work
•	General Structure

</span>

NASA’s ProgPy is an open-source python package supporting research and development of prognostics, health management, and predictive maintenance tools. It implements architectures and common functionality of prognostics, supporting researchers and practitioners.

The goal of this tutorial is to instruct users how to use and extend ProgPy. This tutorial will cover how to use a model, including existing models and additional capabilities like parameter estimation and simulation, as well as how to build a new model from scratch. 

### Definitions and Background



The tutorial will begin with an introduction to prognostics and ProgPy using ProgPy's documentation. Please follow along in the [ProgPy Guide](https://nasa.github.io/progpy/guide.html).

### Tutorial Outline

1. Using an existing model
    - Loading a model
    - Model parameters
    - Simulation
    - Noise
    - Prognostics with data

2. Building a new model 
 
<span style="color:red"> update this as we go along ... </span>


## The Dataset

Let's prepare the dataset that we will use for this tutorial.

In [ ]:
print(desc['description'])

The dataset includes a number of different runs. Let's take a look at the first 10 as a starting point

In [ ]:
print(desc['runs'][:10])

This includes runs from multiple discharges at different kinds. Let's take a look at the trickle discharge run first

In [ ]:
trickle_dataset = data[0]
print(trickle_dataset)
trickle_dataset.plot(y=['current', 'voltage', 'temperature'], subplots=True)

Now let's do the same for a reference discharge run (5)

In [ ]:
reference_dataset = data[5]
reference_dataset.plot(y=['current', 'voltage', 'temperature'], subplots=True)

Now let's take a look at one of the step discharges. This actually includes multiple runs

In [ ]:
print(desc['runs'][7:35])

Includes a charge, which we dont want to include in our characterization. So we should treat that as a different run.

relativeTime resets for each "run". So if we're going to use multiple runs together, we need to stitch these times together.

In [ ]:
data[7]['absoluteTime'] = data[7]['relativeTime']
for i in range(8, 32):
    data[i]['absoluteTime'] = data[i]['relativeTime'] + data[i-1]['absoluteTime'].iloc[-1]

Next we should combine the data into a single dataset and investigate the results

In [ ]:
import pandas as pd
step_dataset = pd.concat(data[7:32], ignore_index=True)
print(step_dataset)
step_dataset.plot(y=['current', 'voltage', 'temperature'], subplots=True)

Finally, let's investigate the random walk discharge

In [ ]:
print(desc['runs'][35:50])

Like the step discharge, we need to stitch together the times and concatenate the data

In [ ]:
data[35]['absoluteTime'] = data[35]['relativeTime']
for i in range(36, 50):
    data[i]['absoluteTime'] = data[i]['relativeTime'] + data[i-1]['absoluteTime'].iloc[-1]

In [ ]:
random_walk_dataset = pd.concat(data[35:50], ignore_index=True)
print(random_walk_dataset)
random_walk_dataset.plot(y=['current', 'voltage', 'temperature'], subplots=True)

Now the data is ready for this tutorial, let's dive into it.

## Using an existing Model

<span style="color:red">
•	Introduce- existing models, data driven tools

•	Battery Model – Electrochemistry EOD
•	Set parameters (characterize) using data - Chetan - script for characterization, confirm that data works for that.
        m = BatteryElectroChemEOD()
        m.estimate_params(dataset1, params=[...])
        m.estimate_params(dataset2, params=[...])
        ...
        

•	Simulate 

-   introducing state estimation and prediction as concepts

•	Setup an example with prognostics & data (see dataset example)

•	Surrogate model – build surrogate model, compare runtime
</span>


The first component of ProgPy are the **Prognostics Models**. Models describe the behavior of the system of interest and how the state of the system evolves with use. ProgPy includes capability for prognostics models to be [physics-based](https://nasa.github.io/progpy/glossary.html#term-physics-based-model) or [data-driven](https://nasa.github.io/progpy/glossary.html#term-data-driven-model).

All prognostics models have the same [format](https://nasa.github.io/progpy/prog_models_guide.html#progpy-prognostic-model-format) within ProgPy. The architecture requires definition of model inputs, states, outputs, and events which come together to create a system model.

ProgPy includes a collection of [included models](https://nasa.github.io/progpy/api_ref/progpy/IncludedModels.html#included-models) which can be accessed through the `progpy.models` package.


### Loading a Model

To illustrate how to use a built-in model, let's use the [Battery Electrochemistry model](https://nasa.github.io/progpy/api_ref/progpy/IncludedModels.html#:~:text=class%20progpy.models.BatteryElectroChemEOD(**kwargs)). This model predicts the end-of-discharge of a Lithium-ion battery based on a set of differential equations that describe the electrochemistry of the system [Daigle et al. 2013](https://papers.phmsociety.org/index.php/phmconf/article/view/2252).



First, import the model from the `progpy.models` package.

In [ ]:
from progpy.models import BatteryElectroChemEOD

Next, let's create a new battery using the default settings:

In [ ]:
batt = BatteryElectroChemEOD()

### Model parameters

Model parameters describe the specific system the model will simulate. For the Electrochemistry model, the default model parameters are for 18650-type Li-ion battery cells. All parameters can be accessed through `batt.parameters`. Let's print out all of the parameters, followed by the specific parameter for the battery's capacity, denoted as `qMax` in this model.

In [ ]:
print(batt.parameters)
print(batt['qMax'])

Parameter values can be configured in various ways. Parameter values can be passed into the constructor as keyword arguments when the model is first instantiated or can be set afterwards, like so:

In [ ]:
batt['qMax'] = 127000
print(batt['qMax'])

In addition to setting model parameter values by hand, ProgPy includes a [parameter estimation](https://nasa.github.io/progpy/prog_models_guide.html#parameter-estimation:~:text=examples.future_loading-,Parameter%20Estimation,-%23) functionality that tunes the parameters of a general model to match the behavior of a specific system. In ProgPy, the `progpy.PrognosticsModel.estimate_params()` method tunes model parameters so that the model provides a good fit to observed data. In the case of the Electrochemistry model, for example, parameter estimation would take the general battery model and configure it so that it more accurately describes a specific battery. The ProgPy documentation includes a [detailed example](https://nasa.github.io/progpy/prog_models_guide.html#parameter-estimation:~:text=See%20the%20example%20below%20for%20more%20details) on how to do parameter estimation.

### Simulation

Once a model has been created, the next step is to simulate it's evolution throughout time. Simulation is the foundation of prediction, but unlike full prediction, simulation does not include uncertainty in the state and other product (e.g., [output](https://nasa.github.io/progpy/glossary.html#term-output)) representation.

*Future Loading*

Most prognostics models have some sort of [input](https://nasa.github.io/progpy/glossary.html#term-input), i.e. a control or load applied to the system that impacts the system state and outputs. For example, for a battery, the current drawn from the battery is the applied load, or input. In this case, to simulate the system, we must define a `future_loading` function that describes how the system will be loaded, or used, throughout time. (Note that not all systems have applied load, e.g. [ThrowObject](https://nasa.github.io/progpy/api_ref/progpy/IncludedModels.html?highlight=thrownobject#progpy.models.ThrownObject), and no `future_loading` is required in these cases.)

ProgPy includes pre-defined [loading functions](https://nasa.github.io/progpy/api_ref/progpy/Loading.html?highlight=progpy%20loading) in `progpy.loading`. Here, we'll implement the built-in piecewise loading functionality. <span style="color:red"> do we want more description of this or just verbal? </span>

In [ ]:
from progpy.loading import Piecewise

future_loading = Piecewise(
        InputContainer=batt.InputContainer,
        times=[600, 900, 1800, 3000],
        values={'i': [2, 1, 4, 2, 3]})

*Simulate to Threshold*

With this in mind, we're ready to simulate our model forward in time using ProgPy's [simulation functionality](https://nasa.github.io/progpy/prog_models_guide.html#simulation).

Physical systems frequently have one or more failure modes, and there's often a need to predict the progress towards these events and the eventual failure of the system. ProgPy generalizes this concept of predicting Remaining Useful Life (RUL) with [events](https://nasa.github.io/progpy/prog_models_guide.html#events) and their corresponding thresholds at which they occur. 


Often, there is interest in simulating a system forward in time until a particular event occurs. ProgPy includes this capability with `simulate_to_threshold()`. 

First, let's take a look at what events exist for the Electrochemistry model.

In [ ]:
batt.events

The only event in this model is 'EOD' or end-of-discharge. The `progpy.PrognosticsModel.event_state()` method estimates the progress towards the event, with 1 representing no progress towards the event and 0 indicating the event has occurred.  The method `progpy.PrognosticsModel.threshold_met()` defines when the event has happened. In the Electrochemistry model, this occurs when the battery voltage drops below some pre-defined value, which is stored in the parameter `VEOD`. Let's see what this threshold value is.

In [ ]:
batt.parameters['VEOD']

With these definitions in mind, let's simulate the battery model until threshold for EOD is met. We'll use the same `future_loading` function as above. 

In [ ]:
options = { #configuration for this sim
    'save_freq': 100,  # Frequency at which results are saved (s)
    'horizon': 8000  # Maximum time to simulate (s) - This is a cutoff. The simulation will end at this time, or when a threshold has been met, whichever is first
    }
results = batt.simulate_to_threshold(future_loading, **options)

Let's visualize the results. Note that the simulation ends when the voltage value hits the VEOD value of 3.0.

In [ ]:
results.inputs.plot(ylabel='Current drawn (amps)')
results.event_states.plot(ylabel='Battery State of Charge')
results.outputs.plot(ylabel= {'v': "voltage (V)", 't': 'temperature (°C)'}, compact= False)

In addition to simulating to threshold, ProgPy also includes a simpler capability to simulate until a particular time, using `simulate_to()`.

### Noise

A key factor in modeling any real-world application is noise. See the ProgPy [noise documentation](https://nasa.github.io/progpy/prog_models_guide.html#noise) for a detailed description of different types of noise and how to include it in the ProgPy architecture. 

### Prognostics with data



<span style="color:red">
https://nasa.github.io/progpy/prog_algs_guide.html#state-estimation-and-prediction-guide

Note to go to documentation 

- download data from real datasets (from dataset.py)
- each dataset split into many runs; string together a bunch and take this as a single run 
- Plot what load looks like; run through state estimation and prediction and compare to data 
- play around with uncertainty to get bounds right
</span>  

Now that we have a basic simulation of our model, let's make a prediction using the prognostics capabilities within ProgPy. The two basic components of prognostics are [state estimation and prediction](https://nasa.github.io/progpy/prog_algs_guide.html#state-estimation-and-prediction-guide). ProgPy includes functionality to do both. 

To implement a prognostics example, we first need data from our system. We'll use data from <span style="color:red"> INCLUDE REFERENCE and description </span>.

For the battery electrochemistry model, we'll need to use a [state estimator](https://nasa.github.io/progpy/prog_algs_guide.html#state-estimation) because the model state is not directly measureable, i.e. it has hidden states. We'll use an Unscented Kalman filter and the `estimate` method. ProgPy also includes a Particle Filter and a Kalman Filter.  

First, let's load the necessary imports.

In [ ]:
import numpy as np
from progpy.state_estimators import UnscentedKalmanFilter
from progpy.uncertain_data import MultivariateNormalDist

State estimators require an initial state. To define this, we'll first initialize the model and then define the initial state as a distribution of possible states around this using a multi-variate normal distribution. 

In [ ]:
initial_state = batt.initialize() # Initialize model
x_guess = MultivariateNormalDist(initial_state.keys(), initial_state.values(), np.diag([max(x_i*0.1, 0.1) for x_i in initial_state.values()])) # Define distribution around initial state

With our initial distribution defined, we can now instantiate the state estimator.

In [ ]:
ukf = UnscentedKalmanFilter(batt, x_guess)

With this, we're ready to run the Unscented Kalman Filter. To illustrate how state estimation works, let's estimate one step forward in time. First, we'll extract the measurement at this time. 

In [ ]:
# Define time step based on data
dt = dataset['relativeTime'][1] - dataset['relativeTime'][0]

# Data at time point
z = {'t': dataset['temperature'][1], 'v': dataset['voltage'][1]}

Next, we'll estimate the new state by calling the `estimate` method. 

In [ ]:
# Extract input current from data 
i = {'i': dataset['current'][1]}

# Estimate the new state
ukf.estimate(dt, i, z)
x_est = ukf.x.mean

Finally, let's look at the difference between the estimated state and the true measurement.

In [ ]:
print(f"t: {dt:.2f}\n\tEstimate: {x_est}\n\tTruth: {initial_state}")

Now that we know how to do state estimation, the next key component of prognostics is [prediction](https://nasa.github.io/progpy/prog_algs_guide.html#prediction). ProgPy includes multiple predictors, and we'll implement a Monte Carlo predictor here. Let's load the necessary imports. 

In [ ]:
from progpy.predictors import MonteCarlo

Next, let's add some [process and measurement noise](https://nasa.github.io/progpy/prog_models_guide.html?highlight=noise#noise) into our system, to capture any uncertainties. 

In [ ]:
PROCESS_NOISE = 1e-4            # Percentage process noise
MEASUREMENT_NOISE = 1e-4        # Percentage measurement noise

# Apply process noise to state
batt.parameters['process_noise'] = {key: PROCESS_NOISE * value for key, value in initial_state.items()}

# Apply measurement noise to output
z0 = batt.output(initial_state)
batt.parameters['measurement_noise'] = {key: MEASUREMENT_NOISE * value for key, value in z0.items()}

Next, let's set up our predictor. 

In [ ]:
mc = MonteCarlo(batt)

To perform the prediction, we need to specify a few things, including the number of samples we want to use for the prediction, the step size for the prediction, and the prediction horizon (i.e., the time value to predict to).

In [ ]:
NUM_SAMPLES = 100
STEP_SIZE = 1
PREDICTION_HORIZON = 3200 

With this, we are ready to predict. 

In [ ]:
mc_results = mc.predict(initial_state, future_loading_eqn=future_loading, n_samples=NUM_SAMPLES, dt=STEP_SIZE, horizon = PREDICTION_HORIZON)

Finally, let's visualize our results.

In [ ]:
fig = mc_results.time_of_event.plot_hist(keys='EOD')

Now that we understand the basics of state estimation and prediction, as well as how to implement these concepts within ProgPy, we are ready to do a full prognostics example. We'll use the state estimator and predictor we created above.

First, let's set a few values we'll use in the simulation.

In [ ]:
# Constant values
NUM_SAMPLES = 20
PREDICTION_UPDATE_FREQ = 50     # Number of steps between prediction updates
GROUND_TRUTH = {'EOD': 2780}    # NOTE: need to check this is right for this example
PLOT = True

Next, let's initialize a data structure for storing the results, using the following built-in class:

In [ ]:
profile = ToEPredictionProfile()

<span style="color:red"> NEED TO REDEFINE FUTURE_LOADING </span>

Now we'll perform the prognostics. We'll loop through time, estimating the state at each time step, and making a prediction at the `PREDICTION_UPDATE_FREQ`.

<span style="color:red"> NOTE: the following code doesn't work yet - the relativeTime is not monotonically increasing. I believe it is multiple traces strung together. We need to adjust for this </span>

In [ ]:

# Loop through time
for ind in range(dataset.shape[0]):

    # Extract data
    t = dataset['relativeTime'][ind]
    i = {'i': dataset['current'][ind]}
    z = {'t': dataset['temperature'][ind], 'v': dataset['voltage'][ind]}

    # Perform state estimation 
    ukf.estimate(t, i, z)
    eod = batt.event_state(ukf.x.mean)['EOD']
    print("  - Event State: ", eod)

    # Prediction step (at specified frequency)
    if (ind%PREDICTION_UPDATE_FREQ == 0):
        # Perform prediction
        mc_results = mc.predict(ukf.x, future_loading, t0 = t, n_samples=NUM_SAMPLES, dt=dt)
        
        # Calculate metrics and print
        metrics = mc_results.time_of_event.metrics()
        print('  - ToE: {} (sigma: {})'.format(metrics['EOD']['mean'], metrics['EOD']['std']))

        # Save results
        profile.add_prediction(t, mc_results.time_of_event)

With our prognostics results, we can now calculate some metrics to analyze the accuracy. 

First, some imports.

In [ ]:
from progpy.uncertain_data.uncertain_data import UncertainData
from progpy.metrics import samples as metrics

We'll start by calculating the cumulative relative accuracy given the ground truth value. 

In [ ]:
cra = profile.cumulative_relative_accuracy(GROUND_TRUTH)
print(f"Cumulative Relative Accuracy for 'EOD': {cra['EOD']}")

We'll also generate some plots of the results.

In [ ]:
playback_plots = profile.plot(GROUND_TRUTH, ALPHA, True)

<span style="color:red"> Could include some other metrics like prognostics horizon, alpha-lambda, etc., if desired </span>

## Building a new model

In the last sections we described how to tune and use a prognostics model, using a model distributed with ProgPy. However, in many cases a model doesn't yet exist for the system being targeted. In those cases, a new model must be built to describe the behavior and degradation of the system.

In this section we will create a new model from scratch. We will again be using the battery as a target. 

All of the past sections describe how to use an existing model. In this section we will describe how to create a new model. This section specifically describes creating a new physics-based model. NOTE ABOUT SOMETIMES NEEDING TO CREATE A NEW ONE SOMETIME

Physics-based state transition models that cannot be described linearly are constructed by subclassing [progpy.PrognosticsModel](https://nasa.github.io/progpy/api_ref/prog_models/PrognosticModel.html#prog_models.PrognosticsModel). To demonstrate this, we'll create a new model class that inherits from this class. Once constructed in this way, the analysis and simulation tools for PrognosticsModels will work on the new model.
https://nasa.github.io/progpy/prog_models_guide.html#state-transition-models

From here: https://www.sciencedirect.com/science/article/pii/S0951832018301406

In [ ]:
from progpy import PrognosticsModel

$R_{int}(k+1) = R_{int}(k) + w_1(k)$

$SOC(k+1) = SOC(k) - P(k)*\Delta t * E_{crit}(k)^{-1} + w_2(k)$

$E_{crit}(k+1) = E_{crit}(k) + w_3(k)$

w1, 2 and 3 are omitted (process noise, which is covered by ProgPy)

Note: wont actually subclass in practice, but it's to demonstrate

In [ ]:
class SimplifiedEquivilantCircuit(PrognosticsModel):
    inputs = ['P']
    states = [
        'R_int',
        'SOC',
        'E_crit']

In [ ]:
class SimplifiedEquivilantCircuit(SimplifiedEquivilantCircuit):
    default_parameters = {
        'x0': {
            'R_int': 0.027,
            'SOC': 1,
            'E_crit': 202426.858,
        }
    }

In [106]:
class SimplifiedEquivilantCircuit(SimplifiedEquivilantCircuit):
    state_limits = {
        'SOC': (0.0, 1.0)
    }

In [107]:
class SimplifiedEquivilantCircuit(SimplifiedEquivilantCircuit):
    def next_state(self, x, u, dt):
        x['SOC'] = x['SOC'] - u['P'] * dt / x['E_crit']

        return x

$V(k) = v_{oc}(k) - i(k) * R_{int}(k) + \eta (k)$

where

$v_{oc}(k) = v_L - \lambda ^ {\gamma * SOC(k)} - \mu * e ^ {-\beta * \sqrt{SOC(k)}}$

and

$i(k) = \frac{v_{oc}(k) - \sqrt{v_{oc}(k)^2 - 4 * R_{int}(k) * P(k)}}{2 * R_{int}(k)}$

Note that $\eta$ is the measurement noise, which progpy handles, so that's ommitted from the equation below.

Note 2: There is a typo in the paper where the sign of the second term in the $v_{oc}$ term. It should be negative (like above), but is reported as positive in the paper.

In [108]:
class SimplifiedEquivilantCircuit(SimplifiedEquivilantCircuit):
    outputs = ['v']

Note that the input ($P(k)$) is also used in the output, that means it's part of the state of the system. So we will update the states to include this

In [109]:
class SimplifiedEquivilantCircuit(SimplifiedEquivilantCircuit):
    states = [
        'R_int',
        'SOC',
        'E_crit',
        'P']

    def next_state(self, x, u, dt):
        x['SOC'] = x['SOC'] - u['P'] * dt / x['E_crit']
        x['P'] = u['P']

        return x
    

In [119]:
class SimplifiedEquivilantCircuit(SimplifiedEquivilantCircuit):
    default_parameters = {
        'v_L': 11.148,
        'lambda': 0.046,
        'gamma': 3.355,
        'mu': 2.759,
        'beta': 8.482,

        'x0': {
            'R_int': 0.027,
            'SOC': 1,
            'E_crit': 202426.858,
            'P': 0.01  # Added P
        }
    }

In [111]:
import math
class SimplifiedEquivilantCircuit(SimplifiedEquivilantCircuit):
    def output(self, x):
        v_oc = self['v_L'] + self['lambda']**(self['gamma']*x['SOC']) - self['mu'] * math.exp(-self['beta']* math.sqrt(x['SOC']))
        i = (v_oc - math.sqrt(v_oc**2 - 4 * x['R_int'] * x['P']))/(2 * x['R_int'])
        v = v_oc - i * x['R_int']
        return self.OutputContainer({'v': v})

Next lets look at events...

In [112]:
class SimplifiedEquivilantCircuit(SimplifiedEquivilantCircuit):
    events = ['EOD']

In [113]:
class SimplifiedEquivilantCircuit(SimplifiedEquivilantCircuit):
    def event_state(self, x):
        return {'EOD': x['SOC']}

In [120]:
m = SimplifiedEquivilantCircuit()

In [132]:
def future_load(t, x=None):
    if x is None:
        return {'P': 165}
    z = m.output(x)
    return {'P': 15 * z['v']}
results = m.simulate_to_threshold(future_load, dt=0.1, save_freq=0.1)

In [ ]:
fig = results.event_states.plot()

In [ ]:
fig = results.outputs.plot()

### Parameter Estimation

In [ ]:
times = dataset['absoluteTime']
inputs = [elem[1]['voltage'] * elem[1]['current'] for elem in dataset.iterrows()]
outputs = [{'v': elem[1]['voltage']} for elem in dataset.iterrows()]

In [ ]:
def future_load(t, x=None):
    power = np.interp(t, times, inputs)
    return {'P': power}

In [ ]:
dataset

In [ ]:
result = m.simulate_to(30116.56, future_load, dt=1, save_freq=100)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(times, [z for z in dataset['voltage']])
plt.plot(results.times, [z['v'] for z in results.outputs])

In [ ]:
m.parameters

In [ ]:
m['v_L'] = m['v_L']/4

In [ ]:
inputs_reformatted = [{'v': elem[1]['voltage'] * elem[1]['current']} for elem in dataset.iterrows()]
keys = ['v_L', 'lambda', 'gamma', 'mu', 'beta', 'x0']
print('Model configuration before')
for key in keys:
    print("-", key, m[key])
print('Error: ', m.calc_error(times=times.to_list(), inputs=inputs_reformatted, outputs=outputs))


In [ ]:
m.estimate_params(times=times.to_list(), inputs=inputs_reformatted, outputs=outputs, dt=1)

In [ ]:
for key in keys:
    print("-", key, m[key])
print('Error: ', m.calc_error(times=times.to_list(), inputs=inputs_reformatted, outputs=outputs))

In [ ]:
results.outputs.plot()

TODO(CT): Use with prediction

## Advanced Capabilities

### Combination Models

https://nasa.github.io/progpy/prog_models_guide.html#combination-models

This section demonstrates how prognostic models can be combined. There are two times in which this is useful: 

1. When combining multiple models of different inter-related systems into one system-of-system model (i.e., [Composite Models](https://nasa.github.io/progpy/api_ref/prog_models/CompositeModel.html)), or
2. Combining multiple models of the same system to be simulated together and aggregated (i.e., [Ensemble Models](https://nasa.github.io/progpy/api_ref/prog_models/EnsembleModel.html) or [Mixture of Expert Models](https://nasa.github.io/progpy/api_ref/progpy/MixtureOfExperts.html)). This is generally done to improve the accuracy of prediction when you have multiple models that each represent part of the behavior or represent a distribution of different behaviors.

For this example we will combine the competing models using a Mixture of Expert Model

In [ ]:
from progpy import MixtureOfExpertsModel

TODO(CT): COMBINE MODELS FROM ABOVE

TODO(CT): Compare performacne

## Closing

https://nasa.github.io/progpy/index.html#contributing-and-partnering